
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 1 - Search Algorithms <br>
<font color=696880 size=4>
Amir Mohammad Fakhimi


____

# Personal Data

In [ ]:
# Set your student number and name
student_number = '401106663'
Name = "Arian"
Last_Name = "Noori"

# Rules

<font color=red>
Please run all the cells.
</font>

# Libraries

In [1]:
# import libraries here

from abc import abstractmethod
import numpy as np
from numpy import random
import pandas as pd

# Q1: Uninformed Search (DFS, BFS, IDS, A*) (100 Points)

In this jupyter notebook, we aim to implement four uninformed search algorithms: DFS, BFS, IDS, and A*. We also implement a maze class to test the algorithms.

## Tile
Each tile is one maze's square. Each part has a (x, y) coordinate. Also it has g (the cost until now), h (heuristic) and f (sum of g and h).

In [2]:
class Tile:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        self.g = 0
        self.h = 0
        self.f = 0
        
        self.parent = None
        
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __repr__(self):
        return f'({self.x}, {self.y})'

## Maze
It has a map attribute. It's a 2d array of a string type. Each index is one of the following strings:
- '0': A free index which the agent can move to.
- '1': A wall which the agent can't move to.
- 'S': The start point of the agent.
- 'E': The end point of the agent.
- 'V': The tiles that the agent has visited. It is used in the [visualizer website](https://project.amfakhimi.com/maze-visualizer), not in this jupyter notebook.
- 'P': The path that the agent has moved through. It appears after the agent has found the path in the `print_path` function of the `Algorithm` class.

It has three types of initialization:
- a tuple (a, b): a is the height of the map, and b is the width of the map. It generates a random map with 80% of '0' and 20% of '1' and sets the start and end points randomly. It is suggested to use it after setting the `random.seed` to a constant number.
- a string: the path of a csv file which contains the map. You can export your customized map from [here](https://project.amfakhimi.com/maze-visualizer).
- a list or a numpy array: the map itself. It should have 'S' and 'E' in it.

The most important methods are:
- `get_tile`: It returns the value of the tile in the given (x, y) coordinate. Pay attention that the first index is y and the second one is x.
- `export_map`: It exports the map to a csv file. You don't need to use it in this assignment.
- `print_map`: It prints the map in the console to visualize it.

You can create your own map [here](https://project.amfakhimi.com/maze-visualizer).  Before exporting the map, clear the map, first.

In [3]:
class Maze:
    def __init__(self, map, name):
        self.name = name
        
        if type(map) == tuple:
            self.height = map[0]
            self.width = map[1]
            self.map = np.zeros((self.height, self.width), dtype=str)
            self.start = Tile(0, 0)
            self.end = Tile(self.width - 1, self.height - 1)
            
            for y in range(self.height):
                for x in range(self.width):
                    if x == self.start.x and y == self.start.y:
                        self.map[y][x] = 'S'
                    elif x == self.end.x and y == self.end.y:
                        self.map[y][x] = 'E'
                    else:
                        self.map[y][x] = random.choice(['0', '1'], p=[0.8, 0.2])
            
        elif type(map) == str:
            map_csv = pd.read_csv(map, header=None, dtype=str)
            self.map = map_csv.values
            for y, row in enumerate(self.map):
                for x, char in enumerate(row):
                    if char == 'V' or char == 'P':
                        self.map[y][x] = '0'
            
        elif type(map) == list or type(map) == np.ndarray:
            self.map = np.array(map)
        else:
            raise ValueError('Invalid map type')
        
        self.width = len(self.map[0])
        self.height = len(self.map)
        self.initial_map = map
        
        self.start = None
        self.end = None
        for y, row in enumerate(self.map):
            for x, char in enumerate(row):
                if char == 'S':
                    self.start = Tile(x, y)
                elif char == 'E':
                    self.end = Tile(x, y)
                    
        if self.start is None or self.end is None:
            raise ValueError('Map must contain a start and end point')
        
    def set_tile(self, x, y, value):
        if value != 'S' and value != 'E' and value != 'P' and value != '0' and value != '1':
            raise ValueError('Invalid tile value')
        
        self.map[y][x] = value
        
    def get_tile(self, x, y):
        return self.map[y][x]
        
    def export_map(self, file_path):
        csv = pd.DataFrame(self.map)
        if file_path is not None:
            csv.to_csv(file_path, index=False, header=False)
        
        return csv
                
    def print_map(self):
        for row in self.map:
            print(''.join([str(x) for x in row]))

## Algorithms (DFS, BFS, IDS, A*) (80 Points)
It has the following attributes:
- `maze`: The maze object.
- `open_list`: A list of tiles that are not visited yet and we get the tile with lowest f by iterating through it.
- `closed_list`: A list of tiles that are visited.
- `path`: The path that the agent has found.

The most important methods are:
- `get_neighbors` (You should implement it -> 10 points): It returns the neighbors of the given tile.
- `solve` (You should implement it -> 70 points): It solves the maze. You should use and fill the `open_list` and `closed_list` and `path` attributes. You should also determine if there's no path to the end point. It is an abstract method, and you should implement it in the subclasses. DFS has 10 points, BFS has 10 points, IDS has 20 points, and A* has 30 points. It has 70 points in total.
- `print_path`: It prints the map with the path that the agent has found.
- `export_closed_list`: It exports the closed list to a csv file. You don't need to use it in this assignment directly.
- `export_path`: It exports the path to a csv file. You don't need to use it in this assignment directly.

You should implement the `get_neighbors` in `Algorithm` class and `solve` in the subclasses (DFS, BFS, IDS, A*). You are free to add any other methods or attributes to the classes. (10 + 70 = 80 points)

In [4]:
class Algorithm:
    def __init__(self, maze):
        self.maze = maze
        self.open_list = []
        self.closed_list = []
        self.path = []
        self.my_map = [[0 for _ in range(maze.width)] for _ in range(maze.height)]
        for i in range (maze.height):
            for j in range (maze.width):
                self.my_map[i][j] = Tile(j , i)
        pass
    
    
    def clear(self):
        for i in range (len(self.my_map)):
            for j in range (len(self.my_map[i])):
                self.my_map[i][j].parent = None
        self.closed_list = []
    
        
    def get_neighbors(self, tile):
        neighbors = []
        x = tile.x
        y = tile.y
        
        if(x < self.maze.width - 1):
            neighbors.append(self.my_map[y][x + 1])
            #self.my_map[y][x + 1].parent = tile
        
        if(y < self.maze.height - 1):
            neighbors.append(self.my_map[y + 1][x])
            #self.my_map[y + 1][x].parent = tile
       
            
        if(x > 0):
            neighbors.append(self.my_map[y][x - 1])
            #self.my_map[y][x - 1].parent = tile
            
        if(y > 0):
            neighbors.append(self.my_map[y - 1][x])
            #self.my_map[y - 1][x].parent = tile
            
            
        
        return neighbors
    
    def get_tile_from_list(self, tile, tile_list):
        for t in tile_list:
            if t == tile:
                return t
            
        return None
    
    def get_tile_from_open_list(self, tile):
        return self.get_tile_from_list(tile, self.open_list)
    
    def get_tile_from_closed_list(self, tile):
        return self.get_tile_from_list(tile, self.closed_list)
    
    def is_tile_in_list(self, tile, tile_list):
        for t in tile_list:
            if t == tile:
                return True
        return False
    
    def is_tile_in_open_list(self, tile):
        return self.is_tile_in_list(tile, self.open_list)
    
    def is_tile_in_closed_list(self, tile):
        return self.is_tile_in_list(tile, self.closed_list)
    
    def is_tile_walkable(self, tile):
        return self.maze.get_tile(tile.x, tile.y) != '1'
    
    @abstractmethod
    def solve(self):
        pass
    
    def print_path(self, path):
        if path is None:
            print('No path found')
            return
        
        for y, row in enumerate(self.maze.map):
            for x, char in enumerate(row):
                if Tile(x, y) in path and char != 'S' and char != 'E':
                    print('P', end='')
                else:
                    print(char, end='')
            print()
            
    def export_closed_list(self, file_path):
        closed_list = [[]]
        for tile in self.closed_list:
            closed_list[0].append(tile.x)
            closed_list[0].append(tile.y)
            
        csv = pd.DataFrame(closed_list)
        if file_path is not None:
            csv.to_csv(file_path, index=False, header=False)
            
        return csv
        
    def export_path(self, file_path):
        path = [[]]
        for tile in self.path:
            path[0].append(tile.x)
            path[0].append(tile.y)
        
        csv = pd.DataFrame(path)
        if file_path is not None:
            csv.to_csv(file_path, index=False, header=False)
        return csv

In [5]:
class DFS(Algorithm):
    def __init__(self, maze):
        super().__init__(maze)
        
    def solve(self):
        print("Hello DFS!")
        start = self.maze.start
        self.recursive_dfs(start)
        pass
        
    def recursive_dfs(self , start):
        if(self.is_tile_walkable(start) and self.is_tile_in_closed_list(start) == False):
            if(self.maze.get_tile(start.x , start.y) == 'E'):
                self.path.append(start)
                return True
            else:
                self.closed_list.append(start)
                neighbors = self.get_neighbors(start)
                for neighbor in neighbors:
                    if(self.recursive_dfs(neighbor)):
                        neighbor.parent = start
                        self.path.insert(0 , start)
                        return True
        return False

In [31]:
class BFS(Algorithm):
    def __init__(self, maze):
        super().__init__(maze)
        
    def solve(self):
        print("Hello BFS!")
        self.closed_list.append(self.maze.start)
        while(len(self.closed_list) != 0):
            current_tile = self.closed_list[0]
            self.closed_list.remove(current_tile)
            if(self.maze.get_tile(current_tile.x , current_tile.y) == 'E'):
                while(current_tile is not self.maze.start):
                    self.path.insert(0 , current_tile)
                    current_tile = current_tile.parent
                return
            neighbors = self.get_neighbors(current_tile)
            for neighbor in neighbors:
                if(self.is_tile_walkable(neighbor) and neighbor.parent is None and (neighbor.x != 0 or neighbor.y != 0)):
                    neighbor.parent = current_tile
                    self.closed_list.append(neighbor)        
        return

In [32]:
class IDS(Algorithm):
    def __init__(self, maze):
        super().__init__(maze)
        
    def solve(self):
        print("Hello IDS!")
        start = self.maze.start
        index = 1
        while(True):
            if(self.recursive_ids(start , index)):
                return 
            self.clear()
            index += 1

    def recursive_ids(self , start , index):
        if(index == 0):
            return False
        if(self.is_tile_walkable(start) and self.is_tile_in_closed_list(start) == False):
            if(self.maze.get_tile(start.x , start.y) == 'E'):
                self.path.append(start)
                return True
            else:
                self.closed_list.append(start)
                neighbors = self.get_neighbors(start)
                for neighbor in neighbors:
                    if(self.recursive_ids(neighbor , index - 1)):
                        neighbor.parent = start
                        self.path.insert(0 , start)
                        return True
        return False

In [33]:
import math
class AStar(Algorithm):
    def __init__(self, maze):
        super().__init__(maze)
        
    def get_tile_with_lowest_f(self):
        lowest_f = self.open_list[0]
        
        for tile in self.open_list:
            if tile.f < lowest_f.f:
                lowest_f = tile
                
        return lowest_f

    
    def get_h(self, tile):
        return math.sqrt((tile.x - self.maze.end.x) ** 2 + (tile.y - self.maze.end.y) ** 2)

    
    def get_f(self, tile):
        return tile.g + tile.h
        
    def solve(self):
        print("Hello A*!")
        for i in range(self.maze.height):
            for j in range(self.maze.width):
                self.my_map[i][j].h = float('inf')
                self.my_map[i][j].g = float('inf')
                self.my_map[i][j].f = float('inf')
        start = self.maze.start
        start.h = self.get_h(start)
        start.g = 0
        start.f = start.h + start.g
        self.open_list.append(start)
        while(len(self.open_list) != 0):
            current_tile = self.get_tile_with_lowest_f()
            self.open_list.remove(current_tile)
            self.closed_list.append(current_tile)
            if(self.maze.get_tile(current_tile.x , current_tile.y) == 'E'):
                self.create_path(current_tile)
                return
            neighbors = self.get_neighbors(current_tile)
            for neighbor in neighbors:
                if self.is_tile_walkable(neighbor) and (neighbor not in self.open_list) and (neighbor not in self.closed_list):
                    self.open_list.append(neighbor)
                    neighbor.parent = current_tile
                    if(neighbor.f > current_tile.g + 1 + self.get_h(neighbor)):
                        neighbor.g = current_tile.g + 1
                        neighbor.f = neighbor.g + self.get_h(neighbor)
        return



    def create_path(self , tile):
        while(tile is not self.maze.start):
            self.path.insert(0 , tile)
            tile = tile.parent
        pass

The following cell exports all needed data. You can simply run it and upload its output in the [visualizer site](https://project.amfakhimi.com/maze-visualizer).

In [34]:
def export_all(maze, solver, merge_file_path):
    map_csv = maze.export_map(None)
    closed_list_csv = solver.export_closed_list(None)
    path_csv = solver.export_path(None)
    
    merge = pd.concat([closed_list_csv, path_csv, map_csv], axis=0)
    merge.to_csv(merge_file_path, index=False, header=False)

## Test (20 points)
if you want to visualize the map, you can use `export_all` function and upload the output in the [visualizer site](https://project.amfakhimi.com/maze-visualizer).

<br>
<font size=5>
After running below tests with seed 0 and (20, 20) as the maze's size, attach your outputs (4 outputs) to the final answer in Quera.
<br>
Your answers based on your implementations could be different from the expected answers, below. So, don't worry about the correctness of your answer. Just make sure that your code works correctly and you have attached the outputs.

In [38]:
random.seed(0)
maze = Maze((20, 20), 'map')
# maze = Maze('./Map.csv', 'map')

maze.print_map()

S0000000110000100010
11000000100000000001
00000000000001000000
00000001010101000000
00000000001000000001
00001000001010010101
00010000000000000000
01001100101010001100
00001110000000000010
00000100000000100000
00000000001100000000
10100010010000000000
01000000000010010001
00001000000110101100
10100001000001000011
11000000000000000001
00000000000000000000
00000000001000000000
10100100010000000010
0101011100000010100E


In [39]:
solver = DFS(maze)
solver.solve()
path = solver.path

print(path)
solver.print_path(path)
export_all(maze, solver, 'dfs.csv')

Hello DFS!
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (7, 1), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (12, 3), (12, 4), (13, 4), (14, 4), (15, 4), (16, 4), (17, 4), (18, 4), (18, 5), (18, 6), (19, 6), (19, 7), (19, 8), (19, 9), (19, 10), (19, 11), (18, 11), (18, 12), (17, 12), (16, 12), (16, 11), (17, 11), (17, 10), (18, 10), (18, 9), (17, 9), (16, 9), (16, 10), (15, 10), (15, 11), (14, 11), (14, 12), (13, 12), (13, 11), (12, 11), (11, 11), (11, 12), (10, 12), (10, 13), (10, 14), (11, 14), (12, 14), (12, 15), (13, 15), (14, 15), (15, 15), (16, 15), (17, 15), (18, 15), (18, 16), (19, 16), (19, 17), (19, 18), (19, 19)]
SPPPPPPP110000100010
1100000P100000000001
0000000PPPPPP1000000
000000010101P1000000
000000000010PPPPPPP1
000010000010100101P1
000100000000000000PP
0100110010101000110P
0000111000000000001P
0000010000000010PPPP
000000000011000PPPPP
10100010010PPPPPPPPP
0100000000PP1PP1PPP1
0000100000P110101100
1010000100PPP1000011
110000000000PPPPPPP1
000000

In [40]:
solver = BFS(maze)
solver.solve()
path = solver.path

print(path)
solver.print_path(path)
export_all(maze, solver, 'bfs.csv')

Hello BFS!
[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12), (9, 12), (10, 12), (10, 13), (10, 14), (11, 14), (12, 14), (12, 15), (13, 15), (14, 15), (15, 15), (16, 15), (17, 15), (18, 15), (18, 16), (19, 16), (19, 17), (19, 18), (19, 19)]
SPPPPPP0110000100010
110000P0100000000001
000000P0000001000000
000000P1010101000000
000000PP001000000001
0000100P001010010101
0001000P000000000000
0100110P101010001100
0000111PP00000000010
00000100P00000100000
00000000P01100000000
10100010P10000000000
01000000PPP010010001
0000100000P110101100
1010000100PPP1000011
110000000000PPPPPPP1
000000000000000000PP
0000000000100000000P
1010010001000000001P
0101011100000010100E


In [41]:
solver = IDS(maze)
solver.solve()
path = solver.path

print(path)
solver.print_path(path)
export_all(maze, solver, 'ids.csv')

Hello IDS!
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (7, 1), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (12, 3), (12, 4), (11, 4), (11, 5), (11, 6), (10, 6), (9, 6), (9, 7), (9, 8), (10, 8), (10, 9), (9, 9), (9, 10), (8, 10), (8, 11), (8, 12), (9, 12), (10, 12), (10, 13), (10, 14), (11, 14), (12, 14), (12, 15), (13, 15), (14, 15), (15, 15), (16, 15), (17, 15), (18, 15), (18, 16), (19, 16), (19, 17), (19, 18), (19, 19)]
SPPPPPPP110000100010
1100000P100000000001
0000000PPPPPP1000000
000000010101P1000000
00000000001PP0000001
00001000001P10010101
000100000PPP00000000
010011001P1010001100
000011100PP000000010
000001000PP000100000
00000000PP1100000000
10100010P10000000000
01000000PPP010010001
0000100000P110101100
1010000100PPP1000011
110000000000PPPPPPP1
000000000000000000PP
0000000000100000000P
1010010001000000001P
0101011100000010100E


In [42]:
solver = AStar(maze)
solver.solve()
path = solver.path

print(path)
solver.print_path(path)
export_all(maze, solver, 'a_star.csv')

Hello A*!
[(1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4), (5, 4), (5, 5), (6, 5), (6, 6), (7, 6), (7, 7), (7, 8), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12), (9, 12), (10, 12), (10, 13), (10, 14), (11, 14), (12, 14), (12, 15), (13, 15), (14, 15), (15, 15), (16, 15), (16, 16), (17, 16), (17, 17), (18, 17), (19, 17), (19, 18), (19, 19)]
SPP00000110000100010
11P00000100000000001
00PP0000000001000000
000PP001010101000000
0000PP00001000000001
00001PP0001010010101
000100PP000000000000
0100110P101010001100
0000111PP00000000010
00000100P00000100000
00000000P01100000000
10100010P10000000000
01000000PPP010010001
0000100000P110101100
1010000100PPP1000011
110000000000PPPPP001
0000000000000000PP00
00000000001000000PPP
1010010001000000001P
0101011100000010100E
